https://www.openstreetmap.org/copyright

In [1]:
import time
import csv

import osmnx as ox
import networkx as nx

from shapely.geometry import Point, box, Polygon

import geopy.distance as geodesic
import geopandas

from pyproj import CRS

import matplotlib.pyplot as plt
from prettymaps import *

In [2]:
useful_tags_way = ['bridge', 'tunnel', 'oneway', 'lanes', 'ref', 'name',
                    'highway', 'maxspeed', 'service', 'access', 'area',
                    'landuse', 'width', 'est_width', 'junction', 'surface']
ox.utils.config(useful_tags_way=useful_tags_way)
ox.config(use_cache=True, log_console=True, useful_tags_way=useful_tags_way)

2021-11-02 18:02:14 Configured OSMnx 1.0.1
2021-11-02 18:02:14 HTTP response caching is on


In [3]:
park_filter = {"leisure":["park","playground"]}
building_filter = {"building":True}

In [4]:
epsg4326 = CRS.from_epsg(4326)

In [5]:
place_name = "Harlem, New York, USA"
place_name = "Holtsville, New York, USA"

In [6]:
def get_mask_from_place(place_name, mask_region="both"):
    
    area = ox.geocode_to_gdf(place_name)
    area_base = area.to_crs(epsg4326)
    
    w, n = area.bbox_west.asof(0), area.bbox_north.asof(0)
    e, s = area.bbox_east.asof(0), area.bbox_south.asof(0)

    a_w, a_n= area_base.bbox_west.asof(0), area_base.bbox_north.asof(0)
    a_e, a_s = area_base.bbox_east.asof(0), area_base.bbox_south.asof(0)
    
    poly = Polygon( ( (a_w, a_n), (a_e, a_n), (a_e, a_s), (a_w, a_s), (a_w, a_n) ) )

    d = {"idx": ['b'], 'geometry': poly}

    gdf = geopandas.GeoDataFrame(d, crs="EPSG:4326")
    
    use = mask_region

    if use == "street":
        drawing_kwargs = {
                    'background': {'fc': '#000000', 'ec': '#000000', 'hatch': 'ooo...', 'zorder': -1},
                    'perimeter': {'fc': '#000000', 'ec': '#000000', 'lw': 3, 'hatch': 'ooo...', 'hatch_c': '#000000',  'zorder': 0},
                    'green': {'fc': '#000000', 'ec': '#000000', 'hatch_c': '#000000', 'hatch': 'ooo...', 'lw': 0, 'zorder': 1},
                    'water': {'fc': '#000000', 'hatch': 'ooo...', 'hatch_c': '#000000', 'lw': 0, 'zorder': 2},
                    # Street Mask On
                    'streets': {'fc': '#FFFFFF', 'lw': 0, 'zorder': 3},
                    # Building Mask Off
                    'building': {'palette': ['#000000'], 'ec': '#000000', 'hatch': 'ooo...', 'hatch_c': '#000000', 'lw': 0, 'zorder': 4},
                }
    elif use == "building":
        drawing_kwargs = {
                'background': {'fc': '#000000', 'ec': '#000000', 'hatch': 'ooo...', 'zorder': -1},
                'perimeter': {'fc': '#000000', 'ec': '#000000', 'lw': 3, 'hatch': 'ooo...', 'hatch_c': '#000000',  'zorder': 0},
                'green': {'fc': '#000000', 'ec': '#000000', 'hatch_c': '#000000', 'hatch': 'ooo...', 'lw': 0, 'zorder': 1},
                'water': {'fc': '#000000', 'hatch': 'ooo...', 'hatch_c': '#000000', 'lw': 0, 'zorder': 2},
                # Street Mask Off
                'streets': {'fc': '#000000', 'lw': 0, 'zorder': 3},
                # Building Mask On
                'building': {'palette': ['#FFFFFF'], 'ec': '#FFFFFF', 'hatch': 'ooo...', 'hatch_c': '#FFFFFF', 'lw': 0, 'zorder': 4},
            }
    else:
        drawing_kwargs = {
                'background': {'fc': '#000000', 'ec': '#000000', 'hatch': 'ooo...', 'zorder': -1},
                'perimeter': {'fc': '#000000', 'ec': '#000000', 'lw': 3, 'hatch': 'ooo...', 'hatch_c': '#000000',  'zorder': 0},
                'green': {'fc': '#000000', 'ec': '#000000', 'hatch_c': '#000000', 'hatch': 'ooo...', 'lw': 0, 'zorder': 1},
                'water': {'fc': '#000000', 'hatch': 'ooo...', 'hatch_c': '#000000', 'lw': 0, 'zorder': 2},
                # Street Mask On
                'streets': {'fc': '#FFFFFF', 'lw': 0, 'zorder': 3},
                # Building Mask On
                'building': {'palette': ['#FFFFFF'], 'ec': '#FFFFFF', 'hatch': 'ooo...', 'hatch_c': '#FFFFFF', 'lw': 0, 'zorder': 4},
            }
    
    fig, ax = plt.subplots(figsize = (12, 12), constrained_layout = True)

    layers = plot(
        gdf, 

        ax = ax,

        layers = {
                'perimeter': {},
                'streets': {
                    'width': {
                        'motorway': 8,
                        'trunk': 6,
                        'primary': 6,
                        'secondary': 5,
                        'tertiary': 4,
                        'residential': 3,
                        'living_street': 3,
                        'pedestrian': 1.5,
                        'footway': 1.5,
                        'track': 1,
                        'bridleway': 1
                    },
                    'circle': False,
                },
                'building': {'tags': {'building': True}, 'union': False, 'circle': False,},
                'water': {'tags': {'natural': ['water', 'bay']}, 'circle': False,},
                'green': {'tags': {'landuse': 'grass', 'natural': ['island', 'wood'], 'leisure': 'park'}, 'circle': False,},
            },
            drawing_kwargs = drawing_kwargs,

            osm_credit = False #{'x': .02, 'y': .01,'color': '#00000000'}
    )
    
    plt.savefig('../data/' + place_name + '_' + use + '.png') 

In [7]:
# get_mask_from_place(place_name)

In [8]:
import ee
import folium

Run 'ee.Authenticate()' once to let it use your GDrive to save the satellite images.

In [9]:
# ee.Authenticate()

In [10]:
ee.Initialize()

In [11]:
def get_satellite_from_place(place_name):
    area = ox.geocode_to_gdf(place_name)
    area = area.to_crs(epsg4326)
    
    w, n = area.bbox_west.asof(0), area.bbox_north.asof(0)
    e, s = area.bbox_east.asof(0), area.bbox_south.asof(0)
    
    rec = ee.Geometry.BBox(w, s, e, n)
    
    ee_image = (ee.ImageCollection('USDA/NAIP/DOQQ')
            .filter(ee.Filter.date('2019-01-01', '2020-12-31'))
            .filter(ee.Filter.bounds(rec))
            .sort('CLOUDY_PIXEL_PERCENTAGE')
            .mean())
    
    task = ee.batch.Export.image.toDrive(image=ee_image,
                                     scale=0.5,
                                     region=rec,
                                     fileNamePrefix=place_name + '_ee',
                                     maxPixels=1e13,
                                     crs='EPSG:4326',
                                     fileFormat='GEO_TIFF')
    
    task.start()
    
    return task

In [12]:
task = get_satellite_from_place(place_name)

2021-11-02 18:02:22 Retrieved response from cache file "cache/6c8f66530cf2d0e4a2ebda9920b45dcfce4ebf6d.json"
2021-11-02 18:02:22 Created GeoDataFrame with 1 rows from 1 queries


In [13]:
""" Start task only if not yet downloaded. """
# task.start()

' Start task only if not yet downloaded. '

In [14]:
# task.status()

### Run to Download
This will go through the US City List and download the areas that exist on Open Street Maps.

Satellite images get large really quick! <br>
One image can easily be 1GB or more.

In [17]:
with open('CSV_Files/us_cities_states_counties.csv') as f:
    reader = csv.reader(f)
    
    sleep_time = 60  # in seconds
    low = 0
    high = 100
    
    counter = 0
    
    for row in reader:
        
        
        cities = row[0].split('|')
        place = cities[4] + ", " + cities[2] + ", USA"
        
        if counter in range(low, high):
            print("\ncoutner: ", counter)
            print(place)

            try:
                area = ox.geocode_to_gdf(place)
                print("area exists")
                
                new_file = open("satellite_downloaded_cities.csv", 'a')
                new_file.write(place + "\n")
                new_file.close()
                
                # gets the png and downloads satellite images form earth engine
                #get_mask_from_place(place, mask_region="both")
                get_satellite_from_place(place)
                

            except:
                print("area not found")
        
        
        counter += 1
            
        time.sleep(sleep_time)
    
    f.close()



coutner:  10
Bo Guaniquilla, Puerto Rico, USA
area not found2021-11-02 18:06:42 Retrieved response from cache file "cache/dab14dbe666a27efd071457847df55e300827eb7.json"


coutner:  11
Aguada, Puerto Rico, USA
2021-11-02 18:06:42 Retrieved response from cache file "cache/2210112cbd05e9fefe899e89b110bb00a482ca56.json"
area exists
2021-11-02 18:06:42 Created GeoDataFrame with 1 rows from 1 queries

coutner:  12
Ext Los Robles, Puerto Rico, USA
area not found2021-11-02 18:06:42 Retrieved response from cache file "cache/acf48865caa9729a915b08604ad36989b6a70d4b.json"


coutner:  13
Ramey, Puerto Rico, USA
2021-11-02 18:06:42 Retrieved response from cache file "cache/3aa74538e7a750a54b54990d88c54980156fbc16.json"
2021-11-02 18:06:42 Created GeoDataFrame with 1 rows from 1 queries
area exists

coutner:  14
Aguadilla, Puerto Rico, USA
2021-11-02 18:06:42 Retrieved response from cache file "cache/1b907197cfe47d538575004bf0225eb76306facb.json"
area exists

coutner: 2021-11-02 18:06:42 Created Ge

IndexError: list index out of range